# LFM梯度下降算法实现

### 0. 引入依赖

In [2]:
import numpy as np
import pandas as pd

### 1. 数据准备

In [7]:
# 评分矩阵R
R = np.array([[4,0,2,0,1],
             [0,2,3,0,0],
             [1,0,2,4,0],
             [5,0,0,3,1],
             [0,0,1,5,1],
             [0,3,2,4,1],])
len(R[0])

5

### 2. 算法实现

In [12]:
"""
@输入参数：
R：M*N 的评分矩阵
K：隐特征向量维度
max_iter: 最大迭代次数
alpha：步长
lamda：正则化系数

@输出：
分解之后的 P，Q
P：初始化用户特征矩阵M*K
Q：初始化物品特征矩阵N*K
"""

# 给定超参数

K = 5
max_iter = 5000
alpha = 0.0002
lamda = 0.004

# 核心算法
def LFM_grad_desc( R, K=2, max_iter=1000, alpha=0.0001, lamda=0.002 ):
    # 基本维度参数定义
    M = len(R)
    N = len(R[0])
    
    # P,Q初始值，随机生成
    P = np.random.rand(M, K)
    Q = np.random.rand(N, K)
    Q = Q.T
    
    # 开始迭代
    for step in range(max_iter):
        # 对所有的用户u、物品i做遍历，对应的特征向量Pu、Qi梯度下降
        for u in range(M):
            for i in range(N):
                # 对于每一个大于0的评分，求出预测评分误差
                if R[u][i] > 0:
                    eui = np.dot( P[u,:], Q[:,i] ) - R[u][i]
                    
                    # 代入公式，按照梯度下降算法更新当前的Pu、Qi
                    for k in range(K):
                        P[u][k] = P[u][k] - alpha * ( 2 * eui * Q[k][i] + 2 * lamda * P[u][k] )
                        Q[k][i] = Q[k][i] - alpha * ( 2 * eui * P[u][k] + 2 * lamda * Q[k][i] )
        
        # u、i遍历完成，所有特征向量更新完成，可以得到P、Q，可以计算预测评分矩阵
        predR = np.dot( P, Q )
        
        # 计算当前损失函数
        cost = 0
        for u in range(M):
            for i in range(N):
                if R[u][i] > 0:
                    cost += ( np.dot( P[u,:], Q[:,i] ) - R[u][i] ) ** 2
                    # 加上正则化项
                    for k in range(K):
                        cost += lamda * ( P[u][k] ** 2 + Q[k][i] ** 2 )
        if cost < 0.0001:
            break
        
    return P, Q.T, cost

### 3. 测试

In [13]:
P, Q, cost = LFM_grad_desc(R, K, max_iter, alpha, lamda)

print(P)
print(Q)
print(cost)

predR = P.dot(Q.T)

print(R)
predR

[[ 0.3830611   0.82850065  0.82023839  0.52119365  1.07831816]
 [ 0.30047264  1.61805241  0.44803691  0.18957273  0.69562221]
 [ 0.5089245   1.15974588  0.28690078  1.15027101 -0.37415904]
 [ 0.32906774  0.80254889  0.99686519  0.22888021  1.54492518]
 [ 1.33431509  0.33140111  0.74692304  1.149386    0.48054638]
 [ 0.56541894  0.7369649   1.00561082  0.74870477  1.12564774]]
[[ 0.4798044   0.92225918  1.07482479  0.10285196  1.94418819]
 [ 0.20864097  0.47978933  1.07255543  0.48634109  0.91665161]
 [-0.15978907  1.57366588  0.46409477  0.18390879  0.29599675]
 [ 1.49743967  1.06857301  0.64402246  1.7015778   0.39911779]
 [ 0.32778316  0.45598072 -0.24798659  0.34891638  0.44123607]]
0.5421902087209758
[[4 0 2 0 1]
 [0 2 3 0 0]
 [1 0 2 4 0]
 [5 0 0 3 1]
 [0 0 1 5 1]
 [0 3 2 4 1]]


array([[3.97955851, 2.59909912, 2.03827334, 3.30440379, 0.95757906],
       [3.48991137, 2.04939982, 2.94695921, 3.06770057, 1.09826208],
       [1.01301061, 1.18678376, 1.97767619, 3.99406951, 0.86074703],
       [4.99666771, 3.05037658, 2.17238821, 2.99840935, 0.98813726],
       [2.80114856, 2.23799913, 1.00857109, 4.98078217, 1.01632564],
       [4.29729537, 2.9460835 , 2.00696893, 4.00506296, 1.02991035]])